# convert mountainlab output to pyphy

In [1]:
cd ..

/home/droumis/Src/pyphy


In [12]:
from pyphy import tools
import sys, os, json
import pandas as pd
import numpy as np
# from collections import defaultdict
sys.path.append('/home/droumis/Src/franklab_ms4/')
# import ms4_franklab_proc2py as p2p
from mountainlab_pytools import mdaio
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
config_path = '/home/droumis/Src/pyphy/config_virga.json'
config = tools.read_config(config_path)
mda_preproc_dir = '/data2/demetris/JZ1/preprocessing/'
ms_firings_dir = '/data2/demetris/JZ1/mountainlab_output/'

animal = 'JZ1'
dates = list(config['JZ1']['day2date'].values())
dates.sort()
ntrodes = np.arange(1,31)

In [10]:
spikes = tools.get_spikes_from_mda(animal, dates[0:1], ntrodes[0:3], mda_preproc_dir, ms_firings_dir, config_path)
spikes.tail()

animal:JZ1 date:20161114 ntrode:1
applying cluster filter: mua : False
keeping clusters: []
0 firings across clusters
animal:JZ1 date:20161114 ntrode:2
applying cluster filter: mua : False
keeping clusters: [5]
1193976 firings across clusters
animal:JZ1 date:20161114 ntrode:3
applying cluster filter: mua : False
keeping clusters: [45]
2466322 firings across clusters


sampleindex       timedelta
animal day epoch ntrode cluster                             
JZ1    1   7     3      45         314454157 02:54:41.805233
                        45         314468487 02:54:42.282900
                        45         314480801 02:54:42.693366
                        45         314489682 02:54:42.989400
                        45         314502426 02:54:43.414200

In [5]:
spikes.tail()

sampleindex       timedelta
animal day epoch ntrode cluster                             
JZ1    14  7     30     58         350045898 03:14:28.196600
                        58         350046531 03:14:28.217700
                        58         350047029 03:14:28.234300
                        58         350047981 03:14:28.266033
                        18         350049696 03:14:28.323200

In [7]:
tools.save_df_to_pyphy(spikes, 'spikes', config_path)

data_key: spikes, data_cols: ['animal', 'day', 'epoch', 'ntrode', 'timedelta']
saving spikes to /opt/DR_swapdata10/JZ1/pyphy/20161114_JZ1.h5
saving spikes to /opt/DR_swapdata10/JZ1/pyphy/20161115_JZ1.h5
saving spikes to /opt/DR_swapdata10/JZ1/pyphy/20161116_JZ1.h5
saving spikes to /opt/DR_swapdata10/JZ1/pyphy/20161117_JZ1.h5
saving spikes to /opt/DR_swapdata10/JZ1/pyphy/20161118_JZ1.h5
saving spikes to /opt/DR_swapdata10/JZ1/pyphy/20161119_JZ1.h5
saving spikes to /opt/DR_swapdata10/JZ1/pyphy/20161121_JZ1.h5
saving spikes to /opt/DR_swapdata10/JZ1/pyphy/20161122_JZ1.h5
saving spikes to /opt/DR_swapdata10/JZ1/pyphy/20161125_JZ1.h5
saving spikes to /opt/DR_swapdata10/JZ1/pyphy/20161128_JZ1.h5
saving spikes to /opt/DR_swapdata10/JZ1/pyphy/20161129_JZ1.h5
saving spikes to /opt/DR_swapdata10/JZ1/pyphy/20161130_JZ1.h5
saving spikes to /opt/DR_swapdata10/JZ1/pyphy/20161201_JZ1.h5
saving spikes to /opt/DR_swapdata10/JZ1/pyphy/20161202_JZ1.h5


# Marks

In [13]:
date = dates[0]
ntrode = 10

In [19]:
os.path.join(ms_firings_dir, str(date), 'ms4', f'nt{ntrode}')

'/data2/demetris/JZ1/mountainlab_output/20161114/ms4/nt10'

In [20]:
ls '/data2/demetris/JZ1/mountainlab_output/20161114/ms4/nt10'

clips.mda                 firings_raw.mda      metrics_merged_tagged.json
filt.mda.prv              marks.mda            metrics_raw.json
firings_burst_merged.mda  metrics_merged.json  pre.mda.prv


In [21]:
marks_path = os.path.join(ms_firings_dir, str(date), 'ms4', f'nt{ntrode}', 'marks.mda')

In [40]:
marks = mdaio.readmda(marks_path)

In [25]:
marks.shape

(8, 1, 2442735)

In [27]:
firings.shape

(3, 2442735)

In [29]:
spikes.shape

(10259, 2)

so it looks like the marks contains all firings, regardless of whether it was mu or not.. which is intended behavior

In [47]:
marks.shape

(8, 1, 2442735)

In [51]:
fs = 30000
epgap_sec = 300
ntrode_4_offsets = 1
mda_list = tools.get_mda_list(animal, date, ntrode_4_offsets, mda_preproc_dir)
epoch_sample_start, total_samples = tools.get_epoch_offsets(dataset_dir=' ', opts={'mda_list':mda_list})
# add 5 minute gaps between epochs- franklab convention
samples_offset = np.arange(1,total_samples+1)
epoch_sample_start_offset = np.array(epoch_sample_start)
for ioff, soff in enumerate(epoch_sample_start[1:]):
    epoch_sample_start_offset[ioff+1:] += epgap_sec * fs
    samples_offset[soff+1:] += epgap_sec * fs


In [53]:
samples_offset.shape

(340382160,)

In [54]:
firings_path = os.path.join(ms_firings_dir, str(date), 'ms4', f'nt{ntrode}', 'firings_burst_merged.mda')
firings = mdaio.readmda(firings_path)

In [56]:
firings_times = firings[1,:]

In [57]:
firings_timedelta = pd.TimedeltaIndex(firings_times/fs, unit='s', name='time')

In [59]:
firings_timedelta.shape

(2442735,)

In [44]:
marks_dict_df = {}
if marks.any():
    ch_cols = [f'ch{c:>02d}' for c in np.arange(marks.shape[0])]
    cols = ['animal', 'day', 'epoch', 'ntrode', 'timedelta', 'sampleindex']
    marks_dict_df[date, ntrode] = pd.DataFrame(columns=cols+ch_cols)

In [ ]:
    spikes_mda_inds = firings[1,firings_bool].astype(int)
    spikes_sampleindex = samples_offset[spikes_mda_inds]
    spikes_timedelta = pd.TimedeltaIndex(spikes_sampleindex/fs, unit='s', name='time')
    spikes_df[date][ntrode]['sampleindex'] = spikes_sampleindex
    spikes_df[date][ntrode]['timedelta'] = spikes_timedelta
    spikes_df[date][ntrode]['cluster'] = firings[2,firings_bool].astype(int)

    #add animal, day, epoch cols
    spikes_df[date][ntrode]['animal'] = animal
    spikes_df[date][ntrode]['day'] = convert_dates_to_days(animal, date, config_path)
    spikes_df[date][ntrode]['ntrode'] = ntrode
    spikes_df[date][ntrode]['epoch'] = 0
    ep_samp_start_timedelta = pd.TimedeltaIndex([ep_s/fs  for ep_s in epoch_sample_start_offset], unit='s', name='time')
    for epn, ep_samp_td in enumerate(ep_samp_start_timedelta):
        spikes_df[date][ntrode].loc[spikes_df[date][ntrode]['timedelta'] >= ep_samp_td, 'epoch'] = epn+1

# ok i want to refactor.. let's commit first